#Find Top Publications for a Funding Agency

In [1]:
!pip install dimcli -U --quiet

import dimcli
from dimcli.utils import *
import os,sys,time,json
from os.path import exists
import copy
import pandas as pd
import more_itertools

print ("Logging in")
#Use dsl.ini file
dsl=dimcli.login()
dsl=dimcli.Dsl()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.6/240.6 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 39.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 kB 6.8 MB/s eta 0:00:00


Searching config file credentials for default 'live' instance..


Logging in
Dimcli - Dimensions API Client (v1.1)
Connected to: <https://nsf.dimensions.ai/api/dsl> - DSL v2.7
Method: dsl.ini file


In [6]:
# NSF GEO: grid.457836.b
# NSF OAC: grid.457789.0
highlight_pubs=dsl.query(f'''search publications
  where year in [2018:2023] and funders.id="grid.457789.0"
  return publications [id+title+doi+type+journal+times_cited+altmetric+field_citation_ratio+relative_citation_ratio+category_for+supporting_grant_ids]
  sort by field_citation_ratio
  limit 1000''').as_dataframe()
#  return funders''').as_dataframe()

highlight_pubs.head()

Returned Publications: 456 (total = 456)
Time: 0.42s


,id,title,altmetric,category_for,doi,field_citation_ratio,supporting_grant_ids,times_cited,type,journal.id,journal.title,relative_citation_ratio
0,pub.1100285094,GW170817: Joint Constraint on the Neutron Star...,17.0,"[{'id': '80220', 'name': '5101 Astronomical Sc...",10.3847/2041-8213/aaa402,137.23,"[grant.4312501, grant.3851309, grant.4273588, ...",449,article,jour.1052750,The Astrophysical Journal Letters,NaN
1,pub.1110800400,"Binary Neutron Star Mergers: Mass Ejection, El...",15.0,"[{'id': '80220', 'name': '5101 Astronomical Sc...",10.3847/1538-4357/aaf054,97.19,"[grant.6815053, grant.4312501, grant.6481485, ...",318,article,jour.1134140,The Astrophysical Journal,NaN
2,pub.1110271603,Time-domain effective-one-body gravitational w...,1.0,"[{'id': '80220', 'name': '5101 Astronomical Sc...",10.1103/physrevd.98.104052,51.04,"[grant.4273588, grant.2785004, grant.3851309, ...",167,article,jour.1320496,Physical Review D,NaN
3,pub.1107250861,Exploring Fundamentally Three-dimensional Phen...,50.0,"[{'id': '80228', 'name': '5109 Space Sciences'...",10.3847/1538-4357/aadcf7,43.50,"[grant.5302212, grant.3849352, grant.7065118, ...",191,article,jour.1134140,The Astrophysical Journal,NaN
4,pub.1134612712,Numerical Relativity Simulations of the Neutro...,8.0,"[{'id': '80220', 'name': '5101 Astronomical Sc...",10.3847/1538-4357/abc9be,41.33,"[grant.9382751, grant.7508023, grant.6815053, ...",77,article,jour.1134140,The Astrophysical Journal,NaN


In [7]:
highlight_pubs.loc[:,['id','title','doi','journal.title','field_citation_ratio','times_cited','altmetric']]

,id,title,doi,journal.title,field_citation_ratio,times_cited,altmetric
0,pub.1100285094,GW170817: Joint Constraint on the Neutron Star...,10.3847/2041-8213/aaa402,The Astrophysical Journal Letters,137.23,449,17.0
1,pub.1110800400,"Binary Neutron Star Mergers: Mass Ejection, El...",10.3847/1538-4357/aaf054,The Astrophysical Journal,97.19,318,15.0
2,pub.1110271603,Time-domain effective-one-body gravitational w...,10.1103/physrevd.98.104052,Physical Review D,51.04,167,1.0
3,pub.1107250861,Exploring Fundamentally Three-dimensional Phen...,10.3847/1538-4357/aadcf7,The Astrophysical Journal,43.50,191,50.0
4,pub.1134612712,Numerical Relativity Simulations of the Neutro...,10.3847/1538-4357/abc9be,The Astrophysical Journal,41.33,77,8.0
...,...,...,...,...,...,...,...
451,pub.1107568073,A Bayesian Approach to Multivariate Adaptive L...,10.5194/npg-2018-45,EGUsphere,0.00,0,1.0
452,pub.1107306733,The role of chain-stiffness in lattice protein...,10.1063/1.5045482,The Journal of Chemical Physics,NaN,10,2.0
453,pub.1105451046,Tin–Selenium Compounds at Ambient and High Pre...,10.1021/acs.jpcc.8b04881,The Journal of Physical Chemistry C,NaN,13,1.0
454,pub.1101387201,Computational Modeling of Stapled Peptides tow...,10.1021/acs.jpcb.8b01014,The Journal of Physical Chemistry B,NaN,8,1.0


This shows how to get the grant information for any of the publications

In [8]:
#Get the grants for a specific publication. You can use any field
index=64
supporting_grants=highlight_pubs.at[index,"supporting_grant_ids"]
print(supporting_grants)

grants_for_selected_pub=dsl.query(f'''search grants
  where id in {json.dumps(supporting_grants)}
  return grants
  ''').as_dataframe()

grants_for_selected_pub

['grant.7507834', 'grant.5063472', 'grant.5496408', 'grant.3131524', 'grant.3947865', 'grant.6619726', 'grant.3945992']
Returned Grants: 7 (total = 7)
Time: 0.25s


,id,title,active_year,end_date,funder_org_name,funder_orgs,grant_number,language,original_title,start_date,start_year
0,grant.7507834,Modeling Physical Processes in the Solar Wind ...,"[2018, 2019, 2020]",2020-03-31,Directorate for Computer & Information Science...,"[{'acronym': 'NSF CISE', 'city_name': 'Alexand...",1811176,en,Modeling Physical Processes in the Solar Wind ...,2018-04-01,2018
1,grant.6619726,PICK UP IONS HAVE BECOME AN INTEGRAL COMPONENT...,"[2016, 2017, 2018, 2019]",2019-04-30,National Aeronautics and Space Administration,"[{'acronym': 'NASA', 'city_name': 'Washington ...",NNX16AG83G,en,PICK UP IONS HAVE BECOME AN INTEGRAL COMPONENT...,2016-05-01,2016
2,grant.5063472,Modeling Physical Processes in the Solar Wind ...,"[2016, 2017, 2018, 2019]",2019-04-30,Directorate for Computer & Information Science...,"[{'acronym': 'NSF CISE', 'city_name': 'Alexand...",1615206,en,Modeling Physical Processes in the Solar Wind ...,2016-05-01,2016
3,grant.5496408,IN ORDER TO EFFECTIVELY USE THE VOYAGER 1 ANIS...,"[2015, 2016, 2017, 2018]",2018-06-24,National Aeronautics and Space Administration,"[{'acronym': 'NASA', 'city_name': 'Washington ...",NNX15AN72G,en,IN ORDER TO EFFECTIVELY USE THE VOYAGER 1 ANIS...,2015-06-25,2015
4,grant.3947865,THE KEY OBJECTIVE OF THIS PROPOSAL IS TO INVES...,"[2014, 2015, 2016, 2017]",2017-05-29,National Aeronautics and Space Administration,"[{'acronym': 'NASA', 'city_name': 'Washington ...",NNX14AJ53G,en,THE KEY OBJECTIVE OF THIS PROPOSAL IS TO INVES...,2014-05-30,2014
5,grant.3945992,PROBING THE PHYSICS OF THE HELIOPAUSE REGION ...,"[2014, 2015, 2016, 2017]",2017-03-11,National Aeronautics and Space Administration,"[{'acronym': 'NASA', 'city_name': 'Washington ...",NNX14AF43G,en,PROBING THE PHYSICS OF THE HELIOPAUSE REGION ...,2014-03-12,2014
6,grant.3131524,Modeling Heliophysics and Astrophysics Phenome...,"[2012, 2013, 2014, 2015, 2016]",2016-06-30,Directorate for Computer & Information Science...,"[{'acronym': 'NSF CISE', 'city_name': 'Alexand...",1144120,en,Modeling Heliophysics and Astrophysics Phenome...,2012-07-01,2012


Explore concepts in the top publications

In [ ]:
# NSF GEO: grid.457836.b
# NSF OAC: grid.457789.0
highlight_pubs_concepts=dsl.query(f'''search publications
  where year in [2018:2023] and funders.id="grid.457789.0"
  return publications [id+title+doi+type+journal+field_citation_ratio+altmetric+category_for+supporting_grant_ids+concepts_scores]
  sort by field_citation_ratio
  limit 1000''').as_dataframe_concepts()

highlight_pubs_concepts


In [16]:
hpc=highlight_pubs_concepts.query("score_avg > 0.72")
hpc.loc[:,['id','title','doi','journal.title','field_citation_ratio','altmetric','concept']]

,id,title,doi,journal.title,field_citation_ratio,altmetric,concept
2,pub.1100285094,GW170817: Joint Constraint on the Neutron Star...,10.3847/2041-8213/aaa402,The Astrophysical Journal Letters,137.23,17.0,binary neutron star merger GW170817
3,pub.1100285094,GW170817: Joint Constraint on the Neutron Star...,10.3847/2041-8213/aaa402,The Astrophysical Journal Letters,137.23,17.0,neutron star merger GW170817
8,pub.1100285094,GW170817: Joint Constraint on the Neutron Star...,10.3847/2041-8213/aaa402,The Astrophysical Journal Letters,137.23,17.0,kilonova models
28,pub.1110800400,"Binary Neutron Star Mergers: Mass Ejection, El...",10.3847/1538-4357/aaf054,The Astrophysical Journal,97.19,15.0,light r-process elements
29,pub.1110800400,"Binary Neutron Star Mergers: Mass Ejection, El...",10.3847/1538-4357/aaf054,The Astrophysical Journal,97.19,15.0,binary neutron star mergers
...,...,...,...,...,...,...,...
27853,pub.1120966304,Strong Orbital Polarization in a Cobaltate-Tit...,10.1103/physrevlett.123.117201,Physical Review Letters,NaN,75.0,oxide heterostructures
27854,pub.1120966304,Strong Orbital Polarization in a Cobaltate-Tit...,10.1103/physrevlett.123.117201,Physical Review Letters,NaN,75.0,X-ray absorption spectra
27855,pub.1120966304,Strong Orbital Polarization in a Cobaltate-Tit...,10.1103/physrevlett.123.117201,Physical Review Letters,NaN,75.0,strong orbital polarization
27901,pub.1117968691,Improving plankton image classification using ...,10.1002/lom3.10324,Limnology and Oceanography Methods,NaN,4.0,context metadata


In [23]:
hpc=highlight_pubs_concepts.query("concept=='supercomputing'")
hpc.loc[:,['id','title','doi','journal.title','field_citation_ratio','altmetric','concept']]

,id,title,doi,journal.title,field_citation_ratio,altmetric,concept


Export the results to a spreadsheet

In [ ]:
highlight_pubs.to_excel("GEO-TopFCR-Papers-2018-2023.xlsx")